In [1]:
from relativisticpy.workbook.frontends.sympify.sympify import Sympify
from sympy import NDimArray, Symbol, MutableDenseNDimArray
from relativisticpy.core.indices import Indices, Idx
from relativisticpy.core.multi_index_obj import MultiIndexObject
from relativisticpy.gr.metric import Metric, MetricIndices
from relativisticpy.gr.derivative import Derivative
from relativisticpy.gr.connection import Connection
from itertools import product, combinations

In [2]:
metric_comps = Sympify.prs('[[1, 0], [0, r**2]]')
inv_metric_comps = Sympify.prs('[[1, 0], [0, 1/(r**2)]]')
metric_basis = Sympify.prs('[r, theta]')
g_ab = Metric(metric_comps, MetricIndices(Idx('a'), Idx('b')), metric_basis)
g_bc = Metric(metric_comps, MetricIndices(Idx('b'), Idx('c')), metric_basis)
g_be = Metric(metric_comps, MetricIndices(Idx('b'), Idx('e')), metric_basis)
g_ec = Metric(metric_comps, MetricIndices(Idx('e'), Idx('c')), metric_basis)
d_e = Derivative(Indices(Idx('e')))
d_c = Derivative(Indices(Idx('c')))
d_b = Derivative(Indices(Idx('b')))


In [3]:
(d_b*(-1*g_ec) + d_c*g_be + d_e*g_bc).indices.indices # Fix => if user specifies the resulting indices then we can just map them as specified

(_b  , _e  , _c  )

In [4]:
(d_c*g_be + d_e*g_bc + d_b*(-1*g_ec)).indices.indices # Fix => if user specifies the resulting indices then we can just map them as specified

(_c  , _b  , _e  )

In [5]:
d = g_ab.inv*(d_b*(-g_ec))
d.indices.indices

(^a  , _e  , _c  )

In [6]:
res = (g_ab.inv*(d_c*g_be + d_e*g_bc + d_b*(-1*g_ec)))/2
res.components

[[[0, 0], [0, 1/r]], [[0, r], [1/r, 0]]]

In [7]:
a = Connection(Indices(-Idx('e'), Idx('a'), Idx('c')))
a.components

[[[0, 0], [0, 0]], [[0, 0], [0, 0]]]

In [8]:
a[Indices(-Idx('a'), Idx('e'), Idx('c'))] = res

hello


TypeError: Connection.__init__() takes from 2 to 3 positional arguments but 4 were given

In [9]:
a.components

sympy.tensor.array.dense_ndim_array.MutableDenseNDimArray

In [10]:
Indices(-Idx('e'), Idx('a'), Idx('c'))._get_all_repeated_location(Indices(Idx('a'), -Idx('e'), Idx('c')))

[[0], [1], [2]]

In [6]:
Connection(g_ab, Indices(-Idx('e'), Idx('a'), Idx('c')))

[[[0, 0], [0, -r]], [[0, 1/r], [1/r, 0]]]

In [2]:
basis = Sympify.prs('[t, r]')
component = Sympify.prs('[[1, 0], [0, r**2]]')
mtr = MetricIndices(-Idx('a'), -Idx('f'))
idcs = Indices(Idx('s'))
metric = Metric(component, mtr, basis)
d = Derivative(basis, idcs)

In [3]:
ind_A = Indices(Idx('a'),  Idx('b'))
ind_D = Indices(Idx('a'), Idx('b'), Idx('c'), Idx('f'))
ind_B = Indices(-Idx('a'), -Idx('b'))
ind_C = Indices(Idx('a'), Idx('f'))

In [10]:
tensor_a = MultiIndexObject(component, Indices(Idx('c'), Idx('b')), basis)
tensor_b = MultiIndexObject(basis, Indices(Idx('a')), basis)
an = d * tensor_a

In [11]:
an.components

[[[0, 0], [0, 0]], [[0, 0], [0, 2*r]]]

In [24]:
Indices(Idx('a'), Idx('b')).find(-Idx('a'))

In [25]:
def method_decorator(arg1, arg2):
    def actual_decorator(func):
        def wrapper(self, *args, **kwargs):
            print(f"Decorator arguments: {arg1}, {arg2}")
            return func(self, *args, **kwargs)
        return wrapper
    return actual_decorator

class MyClass:
    @method_decorator("hello", "world")
    def my_method(self, h):
        print(f"Inside my_method {h}")

obj = MyClass()
obj.my_method('Test')
# Output:
# Decorator arguments: hello, world
# Inside my_method



Decorator arguments: hello, world
Inside my_method Test


In [26]:
[i for i in NDimArray([Symbol('x'), Symbol('y')])]

[x, y]

In [6]:
class AsyncCounter:
    def __init__(self, limit):
        self.limit = limit
        self.count = 0

    def __aiter__(self):
        return self

    async def __anext__(self):
        if self.count < self.limit:
            i = self.count
            self.count += 1
            return i
        else:
            raise StopAsyncIteration

async def generate():
    async for i in AsyncCounter(4):
        yield ind_B.einsum_product(ind_A).generator([i])

# usage
async for gen in generate():
    print(gen)


<coroutine object MultiIndexBase.einsum_product.<locals>.generator at 0x10a5b1030>
<coroutine object MultiIndexBase.einsum_product.<locals>.generator at 0x10a5b1140>
<coroutine object MultiIndexBase.einsum_product.<locals>.generator at 0x10a5b1030>
<coroutine object MultiIndexBase.einsum_product.<locals>.generator at 0x10a5b1140>


/var/folders/1g/_127qcc13ll334lvd9yl9vdc0000gn/T/ipykernel_71465/2336363028.py:22: RuntimeWarning: coroutine 'MultiIndexBase.einsum_product.<locals>.generator' was never awaited
  async for gen in generate():


In [15]:
class MyClass:
    count = 0  # this is a class variable

    def __init__(self):
        MyClass.count += 1  # increment the count when a new instance is created

    @classmethod
    def print_count(cls):  # cls refers to the class MyClass
        print("Number of instances created: ", cls.count)


# Create instances
a = MyClass()
b = MyClass()
c = MyClass()

# Call class method
MyClass.print_count()  # prints "Number of instances created:  3"


Number of instances created:  3


In [7]:
class MyClass:
    def __new__(cls, *args, **kwargs):
        print(f"Args: {args}")
        print(f"Kwargs: {kwargs}")
        return super().__new__(cls)

    def __init__(self, a, b):
        self.a = a
        self.b = b

MyClass('a', a = 'v')

Args: ('a',)
Kwargs: {'a': 'v'}


TypeError: MyClass.__init__() got multiple values for argument 'a'

In [11]:
'a' in {'a' : 'b'}.keys()

True

In [9]:
[dir(i) for i in p]

NameError: name 'p' is not defined

In [11]:
lst = [True, False, True, False, False]

del lst[2]
lst

[True, False, False, False]

In [12]:
[A[i]*B[j] for (i, j) in A.indices * B.indices]

# 1. The indices.__mul__(indices) return an iterable object [( (n1,n2),(1m,m2) ), ( (n3,n4),(m3, m4) ), ... ]

AttributeError: 'SingleIndex' object has no attribute 'indices'

In [97]:
import itertools as it

class Index:

    def __init__(self, id, dim):
        self.dim = dim
        self.id = id
        self.cov = True

    def __neg__(self):
        self.cov = not self.cov
        return self

    def __iter__(self):
        self.__i = 0
        return self
    
    def __len__(self):
        return self.dim

    def __next__(self):
        if self.__i < self.dim:
            x = self.__i
            self.__i += 1
            return Index(self.id, x)
        else:
            raise StopIteration

class Indices:

    def __init__(self, indices):
        self.indices = indices

    def indices_iterator(self):
        return list(it.product(*[range(i + 1) for i in (len(i) for i in self.indices)]))

    def __iter__(self):
        self.__length = int(len(self.indices_iterator()))
        self.__i = 1
        return self

    def __next__(self):
        if self.__i <= self.__length:
            x = self.__i
            self.__i += 1
            return self.indices_iterator()[x-1]
        else:
            raise StopIteration

    def __mul__(self, other):
        self.self_sum_locations = []
        self.other_sum_locations = []
        [(i, j) for (i, j) in (other, self)]



In [102]:
operator.add

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (3, 0),
 (3, 1),
 (3, 2),
 (3, 3)]

In [93]:
import itertools

def generate_combinations(t):
    # Create a range for each value in the tuple
    ranges = [range(i + 1) for i in t]
    # Use itertools.product to generate combinations
    return list(itertools.product(*[range(i + 1) for i in t]))


(0, 0)
(0, 1)
(0, 2)
(1, 0)
(1, 1)
(1, 2)
(2, 0)
(2, 1)
(2, 2)


In [13]:
a = Value(4)
b = Value(8)

In [14]:
c = b*a

In [15]:
a.value

12

In [75]:
[(i, j) for i, j in product(['a', 'b', 'c'], ['A', 'B', 'C'])]

[('a', 'A'),
 ('a', 'B'),
 ('a', 'C'),
 ('b', 'A'),
 ('b', 'B'),
 ('b', 'C'),
 ('c', 'A'),
 ('c', 'B'),
 ('c', 'C')]